# 1. Clone the repository and install libs

In [1]:
import os

target_path = "E:/OneDrive/1Project/DTI6287/hw2" #change to your own directory
os.chdir(target_path)

!git clone https://github.com/Cycinter/BTCTalkScrapperLabeling
%cd BTCTalkScrapperLabeling

E:\OneDrive\1Project\DTI6287\hw2\BTCTalkScrapperLabeling


Cloning into 'BTCTalkScrapperLabeling'...
C:\Users\cycin\AppData\Roaming\Python\Python312\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
!pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# 2. Test Scrapper
You need to download a Firefox Explorer.

In [3]:
from webdriver_manager.firefox import GeckoDriverManager

# This will download and return the path of the Firefox driver. Then copy the address in the "service =" in  next code block
print(GeckoDriverManager().install())

C:\Users\cycin\.wdm\drivers\geckodriver\win64\v0.36.0\geckodriver.exe


In [4]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options

# Use Firefox in normal (non-headless) mode for debugging
options = Options()
# Do NOT use headless for now
# options.add_argument("--headless")

# Manually specify geckodriver path
service = Service("C:/Users/cycin/.wdm/drivers/geckodriver/win64/v0.36.0/geckodriver.exe")

# Start driver
driver = webdriver.Firefox(service=service, options=options)

# Add an implicit wait to ensure page loads before continuing
driver.implicitly_wait(10)  # wait up to 10 seconds

# Test
driver.get("https://www.python.org")
print(driver.title)

driver.quit()

Welcome to Python.org


# 2. Scrap the raw post from a topic on bitcointalk
Define the topic_id.
e.g. https://bitcointalk.org/index.php?topic=203.0 the topic_id = 203

In [5]:
from get_posts_from_topic import process_topic
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

# change topic ID 
topic_id = 203
posts = process_topic(topic= topic_id, driver=driver)
# driver.quit()

print(f"✅ Retrieved {len(posts)} posts.")


✅ Retrieved 21 posts.


In [6]:
print("\n📝 Sample posts:\n")

for floor in sorted(posts.keys())[:5]:  # just show first 5 posts
    post = posts[floor]
    print(f"Floor: {floor}")
    print(f"User: {post['poster']}")
    print(f"Date: {post['date']}")
    print(f"Content: {post['post']}")
    print("-" * 100)



📝 Sample posts:

Floor: 1
User: Timo Y
Date: 2010-06-23 10:29:37
Content: Here is an idea how Bitcoin could be improved:

In the current implementation, scarcity is created by setting up arbitrary computational problems whose sole purpose is to create scarcity. The computational problems are artificially created by the system, much like a fiat currency. There is no natural demand to solve those problems.

This is macroeconomically suboptimal because a lot of CPU power is being "wasted" instead of being put to good use (opportunity cost). It would be bad for the environment too, if BC were ever to become a world currency.

Why not use those CPU cycles to perform computations with real value? This could be done via some sort of grid computing infrastructure such as BOINC, except that it would be a marketplace instead of a volunteer grid.

For example:
A drug company wishing to model the structure of some protein could use Bitcoins to buy "computing credits". The computing credits can th

In [7]:
import json

# Save raw posts to JSON
with open(f"data/raw_posts_topic{topic_id}.json", "w", encoding="utf-8") as f:
    json.dump(posts, f, ensure_ascii=False, indent=2, default=str)

print(f"✅ Saved {len(posts)} posts to raw_posts_topic{topic_id}.json")

✅ Saved 21 posts to raw_posts_topic203.json


# 3. Extract addresses from user profiles with regularization

In [9]:
import json

# Load the saved posts JSON file
with open(f"data/raw_posts_topic{topic_id}.json", "r", encoding="utf-8") as f:
    posts = json.load(f)

# Convert posts from dict to list for easier iteration
posts_list = []

for floor, post in posts.items():
    posts_list.append({
        "floor": floor,
        "username": post["poster"],
        "poster_id": post["poster_id"],  # ✅ Needed for profile lookup
        "post_date": post["date"],
        "post_content": post["post"]
    })

In [10]:
from get_address_from_user import get_address_from_user

# Store results here
filtered_posts = []

for post in posts_list:
    user_id = int(post["poster_id"])
    addresses = get_address_from_user(driver, user_id)

    if not addresses:
        continue

    for addr in addresses:
        filtered_posts.append({
            "floor": post["floor"],
            "username": post["username"],
            "poster_id": post["poster_id"],
            "address": addr,
            "post_content": post["post_content"]
        })

print(f"✅ Found addresses in {len(filtered_posts)} posts.")

for post in filtered_posts:
    print(f"User ID: {post['poster_id']}")
    print(f"username: {post['username']}")
    print(f"Address: {post['address']}")
    print("-" * 40)


✅ Found addresses in 6 posts.
User ID: 271
username: Timo Y
Address: 1E8ydXZKFLG6zHdcZoRjRT4m4AYgYYjyAi
----------------------------------------
User ID: 143
username: laszlo
Address: 157fRrqAKrDyGHr1Bx3yDxeMv8Rh45aUet
----------------------------------------
User ID: 292
username: BitLex
Address: 1D8AAABEMVaH9AQU1js4ST5YmTvFf8hcyj
----------------------------------------
User ID: 269
username: Bitcoiner
Address: 13Cq8AmdrqewatRxEyU2xNuMvegbaLCvEe
----------------------------------------
User ID: 382
username: Ground Loop
Address: 1HrAmQk9EuH3Ak6ugsw3qi3g23DG6YUNPq
----------------------------------------
User ID: 413
username: bg002h
Address: 1GCDzqmX2Cf513E8NeThNHxiYEivU1Chhe
----------------------------------------


# 4. create raw post and profile for ChatGPT (old module)

In [13]:
import json
import os

# Define input and output file paths
input_path = f"data/raw_posts_topic{topic_id}.json"
output_dir = "BitcoinTalkThreads"
output_path = os.path.join(output_dir, f"{topic_id}.json")  # Saves as e.g. "203.json"

# Create the output folder if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Load the raw post data
with open(input_path, "r", encoding="utf-8") as f:
    raw_posts = json.load(f)

# Start constructing the output structure
thread_json = {
    "thread_number": topic_id,
    "title": f"Topic {topic_id}",
    "thread_url": f"https://bitcointalk.org/index.php?topic={topic_id}",
    "num_messages": len(raw_posts)
}

# Add each post using numeric keys ("0", "1", etc.)
for idx, (floor, post) in enumerate(raw_posts.items()):
    thread_json[str(idx)] = {
        "poster": post["poster"],
        "poster_id": post["poster_id"],
        "date": post["date"],
        "message": post["post"]
    }

# Save the final converted thread to output
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(thread_json, f, indent=2, ensure_ascii=False)

print(f"✅ Saved formatted thread to {output_path}")


✅ Saved formatted thread to BitcoinTalkThreads\203.json


# 5. Run to merge raw json(raw post + raw profile) for ChatGPT and use Chatgpt to extract address (new model)
Update your openai api key in yaml file:
copy file example_conf.yaml.
Change the file name to conf.yaml and update your openai api key in this file.

In [14]:
from get_user_profile_raw import get_raw_profile_from_user
import json

with open(f"data/raw_posts_topic{topic_id}.json", "r", encoding="utf-8") as f:
    posts = json.load(f)

poster_id_map = {}

for msg_id, post in posts.items():
    poster_id = post["poster_id"]
    if poster_id not in poster_id_map:
        profile_info = get_raw_profile_from_user(driver, poster_id)
        poster_id_map[poster_id] = profile_info

for msg_id, post in posts.items():
    poster_id = post["poster_id"]
    post["profile"] = poster_id_map.get(poster_id, {
        "user_id": poster_id,
        "url": f"https://bitcointalk.org/index.php?action=profile;u={poster_id}",
        "raw_text": ""
    })

# Save to data
output_path = f"data/merged_posts_with_profiles_topic{topic_id}.json"
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(posts, f, ensure_ascii=False, indent=2)

print(f"✅ Merged file saved to {output_path}")


✅ Merged file saved to data/merged_posts_with_profiles_topic203.json


In [22]:
!python extract_address_with_chatgpt.py --topic_id=203

Num threads w at least one address / transaction id: 0, num messages: 0
✅ All GPT outputs saved to resps\gpt_outputs_topic203_20250519_2136.json
✅ Matching CSV exported to resps\gpt_outputs_topic203_20250519_2136.csv



100%|██████████| 21/21 [00:17<00:00,  1.22it/s]


In [20]:
pip install --upgrade openai

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip
